# Make ToO commands for asteroids/comets/satelites


In [49]:
import re
from datetime import datetime, timedelta
from pathlib import Path
import glob
import numpy as np
import os

# Static parameters (adjust if needed)
static_params = {
    'filt': 'r+r+r',
    'exp': '480+480+480',
    'noexp': '1+1+1',
    'name': '',
    'p': '5700',
    'obs_type': 'NS',
    'iit_obs': 'sameer',
    'hanle_obs': 'rigzin'
}

def load_template(path):
    text = Path(path).read_text().strip()
    # Ensure it's a single-line template
    text = " ".join(text.split())
    return text

def parse_line(line):
    # Example line format:
    # 2025 10 28 1100  21.1635    +14.549    107.3  19.9    0.30   0.070 290  +52   +10    0.38  047  +30   Map/Offsets
    cols = line.strip().split()
    if len(cols) < 8:
        return None

    # Date and UT (hm)
    year, month, day = cols[0], cols[1], cols[2]
    hm = cols[3]  # e.g., '1100' meaning 11:00:00
    if not re.fullmatch(r"\d{4}", hm):
        return None
    hh, mm = hm[:2], hm[2:]
    obs_dt = datetime.strptime(f"{year}-{month}-{day} {hh}:{mm}:00", "%Y-%m-%d %H:%M:%S")
    iso_time = obs_dt.strftime("%Y-%m-%dT%H:%M:%S")
    ymd = obs_dt.strftime("%Y%m%d")

    # RA in hours, DEC in degrees
    ra_deg = float(cols[4])*15
    dec_deg = float(cols[5])

    # Motion columns: header shows '"/sec  "/sec' so arcsec/sec.
    # make_too_v2.py commonly uses deg/hr; numerically, arcsec/sec equals deg/hr (factor 3600 cancels).
    ra_rate = float(cols[8]) 
    dec_rate = float(cols[9])

    # If signs are attached separately, handle them
    # Try to detect explicit sign tokens preceding numbers
    def signed_value(tokens, idx):
        if tokens[idx] in ["+", "-"]:
            return float(tokens[idx] + tokens[idx+1])
        return float(tokens[idx])

    try:
        # Try sign-aware parse using known approximate indices
        ra_rate = signed_value(cols, 8)
        dec_rate = signed_value(cols, 9)
    except Exception:
        # Fallback to previously parsed floats
        pass

    # Extract magnitude (V) from column 7
    mag = None
    if len(cols) > 7:
        try:
            mag = float(cols[7])
        except (ValueError, IndexError):
            pass

    return {
        "utc_time": iso_time,
        "date_ymd": ymd,
        "ra_deg": ra_deg,
        "dec_deg": dec_deg,
        "ra_rate": ra_rate,   # deg/hr numerically equal to arcsec/sec
        "dec_rate": dec_rate,  # deg/hr numerically equal to arcsec/sec
        "mag": mag
    }

def parse_ephem_file(path):
    lines = Path(path).read_text().splitlines()
    print(f'Number of lines in epherides = {len(lines)}')#\nSample:\n{lines[0]}')
    data = []
    # Skip first two header lines
    for i, line in enumerate(lines):
        print(line)
        if not line.strip():
            continue
        rec = parse_line(line)
        if rec:
            data.append(rec)
            print(f'Appending: {rec}')
    return data

def make_command(template, rec, params):
    cmd = template
    replacements = {
        "PH_RIGHT_ASC": f"{rec['ra_deg']:.8f}",
        "PH_DECLINATION": f"{rec['dec_deg']:.8f}",
        "PH_RATE_RA": f"{rec['ra_rate']:.8f}",
        "PH_RATE_DEC": f"{rec['dec_rate']:.8f}",
        "PH_OBS_TIME_UTC": rec["utc_time"],
        "PH_FILT": params["filt"],
        "PH_EXPOSURE": params["exp"],
        "PH_NAME": params["name"],
        "PH_PRIORITY": params["p"],
        "PH_OBS_TYPE": params["obs_type"],
        "PH_DATE": rec["date_ymd"],
        "PH_IIT_OBS": params["iit_obs"],
        "PH_HANLE_OBS": params["hanle_obs"],
        'PH_NO_EXP': static_params['noexp']
    }
    for k, v in replacements.items():
        cmd = cmd.replace(k, v)
    
    obs_csv = ','.join(replacements.values())

    return cmd, obs_csv

def get_exp_from_magnitude(mag, mag_ref=20, exp_ref=600):
    """
    Calculate exposure time required to detect an object of given magnitude.
    
    Parameters:
    -----------
    mag : float
        Target magnitude
    mag_ref : float, optional
        Reference magnitude (default: 21)
    exp_ref : float, optional
        Reference exposure time in seconds for mag_ref (default: 600)
    
    Returns:
    --------
    float
        Required exposure time in seconds
    
    Formula:
    --------
    t = t_ref * 10^((mag - mag_ref) / 2.5)
    
    This follows the magnitude system where a difference of 2.5 magnitudes
    corresponds to a factor of 10 in brightness.
    """
    exp_time = exp_ref * (10 ** ((mag - mag_ref) / 2.5))
    return exp_time

def get_exp(dec, ra_rate, dec_rate, mag, pix_scale=0.406, max_streak_length=75, exposure_limit=600, mag_ref=21, exp_ref=600):
    """
    Calculate exposure time considering both streak length and magnitude requirements.
    
    Parameters:
    -----------
    dec : float
        Declination in degrees
    ra_rate : float
        RA rate in deg/hr
    dec_rate : float
        DEC rate in deg/hr
    mag : float
        Target magnitude
    pix_scale : float, optional
        Pixel scale in arcseconds per pixel (default: 0.4)
    max_streak_length : float, optional
        Maximum streak length in arcseconds (default: 50)
    exposure_limit : float, optional
        Maximum exposure time limit in seconds (default: 600)
    mag_ref : float, optional
        Reference magnitude (default: 21)
    exp_ref : float, optional
        Reference exposure time in seconds for mag_ref (default: 600)
    
    Returns:
    --------
    float
        Required exposure time in seconds (maximum of streak-based and magnitude-based)
    """
    # Calculate exposure based on streak length
    net_rate = np.sqrt(dec_rate**2 + (ra_rate * np.cos(np.radians(dec))) ** 2)
    exp_time_streak = max_streak_length * pix_scale / net_rate 

    # Calculate exposure based on magnitude
    exp_time_mag = get_exp_from_magnitude(mag, mag_ref, exp_ref)

    print(f'dec={dec}, ra_rate={ra_rate}, dec_rate={dec_rate}, mag={mag}, net_rate={net_rate}, exp_time_streak={exp_time_streak}, exp_time_mag={exp_time_mag}')
    if exp_time_mag < exp_time_streak: return np.minimum(exp_time_streak, exposure_limit)
    else: return -1

TEMPLATE_FILE = "maketoo_template"
date = datetime.now().strftime('%y%m%d')
date = '251217'
TODAYS_DIR = f"data/{date}"
TODAYS_FILE = f"{TODAYS_DIR}{os.sep}{date}.txt"
OUTPUT_FILE = f"{TODAYS_DIR}/generated_too_commands.sh"
OBS_TABLE_FILE = "obs_summary.csv"
template = load_template(TEMPLATE_FILE)
print(f'TODAYS_DIR={TODAYS_DIR}')
print(f'OUTPUT_FILE={OUTPUT_FILE}')

# Function to check if a line contains a single word with 7 alphanumeric characters (object name)
def is_object_name(line):
    words = line.strip().split()
    if len(words) == 1:
        word = words[0]
        # Check if it's exactly 7 alphanumeric characters
        if len(word) == 7 and word.isalnum():
            return True
    return False

# Read TODAYS_FILE line by line and process objects
name = None
ephemeris_lines = []
all_commands = []
obs_table = []

with open(TODAYS_FILE, 'r') as f:
    for line in f:
        line_stripped = line.strip()
        
        # Check if this line is an object name
        if is_object_name(line_stripped):
            # If we have a previous object, process its ephemerides
            if name is not None and ephemeris_lines:
                print(f'## {name} ##')
                print('#############')
                static_params['name'] = name
                
                # Parse all ephemeris lines for this object
                obs_list = []
                for ephem_line in ephemeris_lines:
                    rec = parse_line(ephem_line)
                    if rec:
                        obs_list.append(rec)
                
                if obs_list:
                    # Choose the first line with UTC timestamp greater than current UTC
                    # If there's only one line, choose that
                    current_utc = datetime.utcnow()
                    target_time = current_utc #+ timedelta(minutes=120)
                    
                    if len(obs_list) == 1:
                        rec = obs_list[0]
                    else:
                        # Find the first line with UTC timestamp >= target_time
                        rec = None
                        for obs in obs_list:
                            obs_time = datetime.strptime(obs['utc_time'], '%Y-%m-%dT%H:%M:%S')
                            if obs_time >= target_time:
                                rec = obs
                                break
                        # If no line found, use the first one
                        if rec is None:
                            rec = obs_list[0]
                    
                    # Calculate exposure time (use magnitude if available, otherwise default to 20)
                    mag = rec.get('mag', 20)
                    exp_time = get_exp(dec=rec['dec_deg'], ra_rate=rec['ra_rate'], dec_rate=rec['dec_rate'], mag=mag)
                    
                    if exp_time > 0:
                        # Update static_params with calculated exposure time
                        static_params['exp'] = f'{int(exp_time)}+{int(exp_time)}+{int(exp_time)}'
                        cmd, obs_csv = make_command(template, rec, static_params)
                        all_commands.append(cmd)
                        obs_table.append(obs_csv)
                        print(cmd)
                    else:
                        print(f'Skipping due to exp time: {rec}')
                else:
                    print("No observations parsed.")
            
            # Set new object name and reset ephemeris lines
            name = line_stripped
            ephemeris_lines = []
        else:
            # This is not an object name line
            # If we have a current object, check if this is an ephemeris line
            if name is not None:
                rec = parse_line(line_stripped)
                if rec:
                    # This is a valid ephemeris line, add it to the list
                    ephemeris_lines.append(line_stripped)

# Process the last object if there is one
if name is not None and ephemeris_lines:
    print(f'## {name} ##')
    print('#############')
    static_params['name'] = name
    
    # Parse all ephemeris lines for this object
    obs_list = []
    for ephem_line in ephemeris_lines:
        rec = parse_line(ephem_line)
        if rec:
            obs_list.append(rec)
    
    if obs_list:
        # Choose the first line with UTC timestamp at least 10 minutes greater than current UTC
        # If there's only one line, choose that
        current_utc = datetime.utcnow()
        target_time = current_utc + timedelta(minutes=10)
        
        if len(obs_list) == 1:
            rec = obs_list[0]
        else:
            # Find the first line with UTC timestamp >= target_time
            rec = None
            for obs in obs_list:
                obs_time = datetime.strptime(obs['utc_time'], '%Y-%m-%dT%H:%M:%S')
                if obs_time >= target_time:
                    rec = obs
                    break
            # If no line found that's 10 minutes ahead, use the first one
            if rec is None:
                rec = obs_list[0]
        
        # Calculate exposure time (use magnitude if available, otherwise default to 20)
        mag = rec.get('mag', 20)
        exp_time = get_exp(dec=rec['dec_deg'], ra_rate=rec['ra_rate'], dec_rate=rec['dec_rate'], mag=mag)
        
        if exp_time > 0:
            # Update static_params with calculated exposure time
            static_params['exp'] = f'{int(exp_time)}+{int(exp_time)}+{int(exp_time)}'
            cmd, obs_csv = make_command(template, rec, static_params)
            all_commands.append(cmd)
            obs_table.append(obs_csv)
            print(cmd)
        else:
            print(f'Skipping due to exp time: {rec}')
    else:
        print("No observations parsed.")

# Write all commands to output file
with open(OUTPUT_FILE, 'w') as f:
    f.write("\n".join(all_commands) + "\n")

with open(OBS_TABLE_FILE, 'a') as f:
    f.write("\n".join(obs_table) + "\n")

print(f"\nTotal commands generated: {len(all_commands)}")
print(f"Commands written to: {OUTPUT_FILE}")


TODAYS_DIR=data/251217
OUTPUT_FILE=data/251217/generated_too_commands.sh
## ZTF109h ##
#############
dec=35.823, ra_rate=-2.49, dec_rate=1.11, mag=18.9, net_rate=2.303978264583927, exp_time_streak=13.21627051264693, exp_time_mag=86.72638624475555
Skipping due to exp time: {'utc_time': '2025-12-17T21:00:00', 'date_ymd': '20251217', 'ra_deg': 62.919000000000004, 'dec_deg': 35.823, 'ra_rate': -2.49, 'dec_rate': 1.11, 'mag': 18.9}
## C45GWA1 ##
#############
dec=29.803, ra_rate=1.06, dec_rate=1.15, mag=18.9, net_rate=1.4725960152401765, exp_time_streak=20.67776884146579, exp_time_mag=86.72638624475555
Skipping due to exp time: {'utc_time': '2025-12-17T22:50:00', 'date_ymd': '20251217', 'ra_deg': 180.702, 'dec_deg': 29.803, 'ra_rate': 1.06, 'dec_rate': 1.15, 'mag': 18.9}
## ZTF109X ##
#############
dec=28.813, ra_rate=0.1, dec_rate=0.036, mag=19.7, net_rate=0.09472707099779137, exp_time_streak=321.4498208300979, exp_time_mag=181.19710322412087
python ~/growth-scripts/nightly_observation/mak

/var/folders/wz/tf9bd1d17kxcg2r9f2875tv00000gn/T/ipykernel_37178/2827695400.py:243: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.utcnow()


In [43]:
import re

def sexagesimal_to_deg(h, m, s, is_ra=False):
    """Convert sexagesimal (h/m/s or d/m/s) to decimal degrees.
       Normalizes overflow in seconds/minutes.
    """
    # Normalize seconds
    if s >= 60:
        m += int(s // 60)
        s = s % 60

    # Normalize minutes
    if m >= 60:
        h += int(m // 60)
        m = m % 60

    value = h + m/60 + s/3600

    # RA is in hours — convert to degrees
    if is_ra:
        value *= 15

    return value


def parse_coord(coord_str):
    """Parse an RA/Dec string and return (ra_deg, dec_deg)."""

    # Extract numbers including sign
    nums = re.findall(r'[+-]?\d+(?:\.\d+)?', coord_str)
    if len(nums) < 6:
        raise ValueError("Need at least 6 numeric components (H M S D M S).")

    # RA components
    ra_h = float(nums[0])
    ra_m = float(nums[1])
    ra_s = float(nums[2])

    # Dec components
    dec_d = float(nums[3])
    dec_m = float(nums[4])
    dec_s = float(nums[5])

    # Determine sign of Dec
    sign = -1 if coord_str.strip()[0] == '-' or coord_str.split()[3].startswith('-') else 1

    # Convert RA and Dec
    ra_deg = sexagesimal_to_deg(ra_h, ra_m, ra_s, is_ra=True)

    dec_deg = abs(dec_d) + dec_m/60 + dec_s/3600
    dec_deg *= -1 if dec_d < 0 else 1

    return ra_deg, dec_deg

def deg_to_sexagesimal(ra_deg, dec_deg):
    """
    Convert RA and Dec in decimal degrees to a formatted sexagesimal string:
    'HH MM SS.ss ±DD MM SS.s'
    """

    # ---------- RA ----------
    ra_hours = ra_deg / 15.0
    H = int(ra_hours)
    M = int((ra_hours - H) * 60)
    S = (ra_hours - H - M/60) * 3600

    # ---------- DEC ----------
    sign = "+" if dec_deg >= 0 else "-"
    dec_deg_abs = abs(dec_deg)

    D = int(dec_deg_abs)
    DM = int((dec_deg_abs - D) * 60)
    DS = (dec_deg_abs - D - DM/60) * 3600

    # Format with fixed widths / decimal places
    ra_str = f"{H:02d} {M:02d} {S:05.2f} "
    dec_str = f"{sign}{D:02d} {DM:02d} {DS:04.2f}"

    return f"{ra_str}{dec_str}"


def parse_residual(residual_str):

    """
    Parse a string like '4.0-  1.9+' where the sign is trailing.
    Returns two floats with the sign applied.
    """
    # Extract patterns like 4.0- or 1.9+
    tokens = re.findall(r'(\d+(?:\.\d+)?)([+-])', residual_str)
    if len(tokens) != 2:
        raise ValueError("Expected two signed numbers with trailing signs.")

    values = []
    for num_str, sign in tokens:
        value = float(num_str)
        if sign == '-':
            value = -value
        values.append(value)

    return values[0], values[1]

# Apply adjustment on decial coordinates
def adjust(ra, dec, ra_residual, dec_residual):
    ra_delta = -ra_residual/3600.0
    dec_delta = -dec_residual/3600.0
    ra_adj = ra + ra_delta
    dec_adj = dec + dec_delta
    return ra_adj, dec_adj

# RA/DEC fix
def fix(ra_dec_str, residual_str):
    ra, dec = parse_coord(ra_dec_str)
    ra_residual, dec_residual = parse_residual(residual_str)
    ra, dec = adjust(ra, dec, ra_residual, dec_residual)

    return deg_to_sexagesimal(ra, dec)

print(fix("08 16 43.19 +17 43 10.7", "3.8+  .26+"))


08 16 42.94 +17 42 44.70
